In [ ]:
import sys, os
from pathlib import Path
cwd = os.getcwd()
parent = str(Path(cwd).parent)
sys.path.append(parent)

import torch
import gpytorch
from gpytorch.kernels import RBFKernel
from kernels.experimental_uncertainty_kernel import ExperimentalUncertaintyKernel, fix_lengthscale
from copy import deepcopy



### **Test the arbitrary block-size branch, training mode**

In [ ]:
SRBFKernel = fix_lengthscale(RBFKernel)

base_kernel = SRBFKernel()

train_x = torch.randn(10,1)
e1 = torch.Tensor([[1,2]])
e2 = 2*e1 ; e3 = 3*e1 ; e4 = 4*e1
X = torch.vstack((e1,e2,e2,e3,e3,e3,e4,e4,e4,e4))
train_x = torch.hstack((train_x, X))

euk = ExperimentalUncertaintyKernel(base_kernel, train_x, (1,2,3,4), (1,2), outputscale_fn=True, lengthscale_fn=True)
euk.initialize(os_v = torch.Tensor([0.1,0.2]))
euk.initialize(ls_v = torch.Tensor([0.0, 0.1]))

c = euk(train_x, train_x)
print(c.to_dense())

### **Now test the test-test and train-test modes**

In [ ]:
test_x = torch.randn(4,1)
e5 = 5*e1
X = torch.vstack((e5,e5,e5,e5))
test_x = torch.hstack((test_x, X))

cc = euk(test_x, test_x)
print(cc.to_dense())

ccc = euk(train_x, test_x)
print(ccc.to_dense())

### **Test the batched mode with all-equal blocks**

In [ ]:
train_x = torch.randn(9,1)
X = torch.vstack((e1,e1,e1,e2,e2,e2,e3,e3,e3))
train_x = torch.hstack((train_x, X))

euk = ExperimentalUncertaintyKernel(base_kernel, train_x, 3, (1,2), outputscale_fn=True, lengthscale_fn=True)
euk.initialize(os_v = torch.Tensor([0.1,0.2]))
euk.initialize(ls_v = torch.Tensor([0.0, 0.1]))

c = euk(train_x, train_x)
print(c.to_dense())

### **Test-Train and Test-Test cases**

In [ ]:
cc = euk(test_x, test_x)
print(cc.to_dense())

ccc = euk(train_x, test_x)
print(ccc.to_dense())